## Suites - A group of 

##### Import necessary libraries for processing, modelling and trainnig

In [2]:
# Import necessary libraries
import numpy as np
import pandas as pd
# from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [3]:
%pip install xlrd

Note: you may need to restart the kernel to use updated packages.


In [4]:
# load data 
df = pd.read_excel("datasets/concept_drift/default of credit card clients.xls")

In [5]:
df = df.drop(df.index[0])
df = df.rename(columns={"Unnamed: 0": "ID"})
df.head()

,ID,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X15,X16,X17,X18,X19,X20,X21,X22,X23,Y
1,1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
2,2,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
3,3,90000,2,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
4,4,50000,2,2,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
5,5,50000,1,2,1,57,-1,0,-1,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


In [6]:
# features and labels
Xfeatures = df.drop(["Y","ID"],axis=1)
ylabels = df.iloc[ :, -1:]

In [7]:
Xfeatures.columns

Index(['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10', 'X11',
       'X12', 'X13', 'X14', 'X15', 'X16', 'X17', 'X18', 'X19', 'X20', 'X21',
       'X22', 'X23'],
      dtype='object')

In [8]:
# split the dataset
x_train, x_test, y_train, y_test = train_test_split(Xfeatures,ylabels,test_size=0.30,random_state=23)
y_train = y_train.astype('int')
y_test = y_test.astype('int')

In [9]:
# build the model
pipe_lr = Pipeline(steps=[('sc', StandardScaler()),('lr',LogisticRegression())])

In [10]:
pipe_lr

Pipeline(steps=[('sc', StandardScaler()), ('lr', LogisticRegression())])

In [11]:
# Train to Fit
pipe_lr.fit(x_train,y_train)

c:\Users\hirushau\AppData\Local\anaconda3\envs\py310\lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Pipeline(steps=[('sc', StandardScaler()), ('lr', LogisticRegression())])

In [12]:
# Accuracy 
# print(y_test)
pipe_lr.score(x_test,y_test)

0.8122222222222222

#### Using DeepChecks for offline ML data drift detection

In [13]:
import deepchecks

#### Full Suite
+ Data Drift Detection
+ Model Perforamance
+ Data Integrity check
+ Label Ambiguity
+ Other checks

In [14]:
from deepchecks.suites import full_suite

c:\Users\hirushau\AppData\Local\anaconda3\envs\py310\lib\site-packages\deepchecks\suites.py:21: DeprecationWarning:

Ability to import tabular suites from the `deepchecks.suites` is deprecated, please import from `deepchecks.tabular.suites` instead



In [15]:
# create the Dataset Objects
ds_train = deepchecks.Dataset(df=x_train,label=y_train,cat_features=[])
ds_test = deepchecks.Dataset(df=x_test,label=y_test,cat_features=[])

c:\Users\hirushau\AppData\Local\anaconda3\envs\py310\lib\site-packages\deepchecks\__init__.py:136: DeprecationWarning:

Ability to import base tabular functionality from the `deepchecks` package directly is deprecated, please import from `deepchecks.tabular` instead

c:\Users\hirushau\AppData\Local\anaconda3\envs\py310\lib\site-packages\deepchecks\__init__.py:136: DeprecationWarning:

Ability to import base tabular functionality from the `deepchecks` package directly is deprecated, please import from `deepchecks.tabular` instead



In [16]:
#Create the suite 
fsuite = full_suite()

In [1]:
results = fsuite.run(train_dataset=ds_train,test_dataset=ds_test,model=pipe_lr)

NameError: name 'fsuite' is not defined

In [ ]:
results

Accordion(children=(VBox(children=(HTML(value='\n<h1 id="summary_TQBMVKO7VD29QBP0HZCMGOFRA">Full Suite</h1>\n<…

### Feature Drift 

In [ ]:
from deepchecks.suites import data_integrity
data_integrity_suit = data_integrity(ignore_columns=['Y','ID'])

In [ ]:
results = data_integrity_suit.run(model=pipe_lr,train_dataset=ds_train,test_dataset=ds_test)

deepchecks - WARNING - Cannot use model's built-in feature importance on a Scikit-learn Pipeline, using permutation feature importance calculation instead
deepchecks - INFO - Calculating permutation feature importance. Expected to finish in 62 seconds


In [ ]:
results.show()

Accordion(children=(VBox(children=(HTML(value='\n<h1 id="summary_WXPKW0FZVL2QDOOSLXNAT0CWJ">Data Integrity Sui…

In [ ]:
from deepchecks.tabular.checks import FeatureDrift
check = FeatureDrift()
result = check.run(train_dataset=ds_train, test_dataset=ds_test)

In [ ]:
result.show()

In [ ]:
from deepchecks.tabular.checks import MultivariateDrift
check = MultivariateDrift()
result = check.run(train_dataset=ds_train, test_dataset=ds_test)
result.show()

In [ ]:
from deepchecks.tabular.checks import LabelDrift
check = LabelDrift()
result = check.run(train_dataset=ds_train, test_dataset=ds_test)
result.show()

In [ ]:
from deepchecks.tabular.checks import PredictionDrift
check = PredictionDrift()
result = check.run(train_dataset=ds_train, test_dataset=ds_test, model=pipe_lr)
result.show()